In [2]:
from edward.util import Progbar
from keras.layers import Dense
from observations import mnist
import edward as ed
import numpy as np
import os
import tensorflow as tf
from edward.models import Bernoulli, Normal
from utils import generator

Using TensorFlow backend.


In [6]:
sess = tf.InteractiveSession()
d = 2
M = 100 
(x_train, _), (x_test, _) = mnist('./data')
x_train_generator = generator(x_train, M)
print(x_test.shape)

(10000, 784)


In [8]:
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))

hidden = Dense(256, activation='relu')(z.value())
x = Bernoulli(logits=Dense(28 * 28)(hidden))


x_ph = tf.placeholder(tf.int32, [M, 28 * 28])
hidden = Dense(256, activation='relu')(tf.cast(x_ph, tf.float32))
qz = Normal(loc=Dense(d)(hidden),
            scale=Dense(d, activation='softplus')(hidden))

In [9]:
inference = ed.KLqp({z: qz}, data={x: x_ph})
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)